### Import libraries

In [1]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                             drop_user_features=True,
                             drop_expert=True)

In [4]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                       drop_user_features=True,
                       drop_expert=False)

In [5]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              MFCC_std11  MFCC_std12  \
subject                              file_id                           
008ba489-31ad-44d8-856b-fcf72369dc46 0         12.109140   11.702864   
                                     1         10.000873    8.967152   
                                     2          8.318832    8.688411   
                                     3         13.294838    8.694027   
                                     4         12.814457    8.501382   

                                              Crest_Factor  Cough_Length  \
subject                              file_id                               
008ba489-31ad-44d8-856b-fcf72369dc46 0            7.649371      0.767250   
                                     1            8.001334      0.568917   
                                     2           11.189755      0.581667   
                                     3            7.840153      0.540250   
                                     4            7.869070      0.546333   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
               

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [6]:
X_fine = classic_preprocessing(X_fine, stop=None, dummy=False)

In [7]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = classic_preprocessing(X_e, stop=None, dummy=False)
X_e['Expert'] = expert.values

### Split expert models

In [8]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

In [ ]:
DECISION_METRIC = 'roc_auc_score'

#### 1. Logistic regression

In [9]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score  \
max_iter oversampling                                            
10000    True          0.424358       0.579228        0.583072   
         False         0.116406       0.522802        0.733619   
100000   True          0.424358       0.579228        0.583072   
         False         0.116406       0.522802        0.733619   

                       explained_variance_score  
max_iter oversampling                            
10000    True                         -0.942598  
         False                        -0.065700  
100000   True                         -0.942598  
         False                        -0.065700

In [10]:
# Best model parameters

best_log = log_results.iloc[[
    log_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.424358,0.579228,0.583072,-0.942598


In [11]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LOGISTIC_PARAMS)

In [12]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.365471,0.655199,0.69919,-1.158433


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,True,0.326309,0.571407,0.614173,-1.281943


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_iter,oversampling,,,,
10000,False,0.611063,0.644425,0.647724,-0.410186


#### 2. Linear Discriminant Analysis

In [13]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.427548,0.581468,0.582882,-0.935633
False,0.122605,0.524261,0.733904,-0.068523


In [14]:
# Best model parameters

best_lda = lda_results.iloc[[
    lda_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.427548,0.581468,0.582882,-0.935633


In [15]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LDA_PARAMS)

In [16]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.37692,0.669532,0.692707,-1.152849


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.326972,0.571416,0.607874,-1.296011


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
False,0.610512,0.645963,0.649655,-0.400898


#### 3. K-nearest Neighbors

In [17]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.499423       0.646364        0.621113   
            False         0.468613       0.637330        0.719734   
2           True          0.492222       0.645516        0.668379   
            False         0.276739       0.566098        0.742748   
3           True          0.489337       0.629823        0.557775   
            False         0.405214       0.606831        0.728293   
4           True          0.485425       0.630811        0.594864   
            False         0.267270       0.562644        0.741227   
5           True          0.479726       0.614988        0.519829   
            False         0.355189       0.588455        0.734665   
6           True          0.479520       0.619738        0.553971   
            False         0.240226       0.555529        0.741797   
7           True          0.468249       0.598287        0.490062   
            False         0.304540       0.570866        0.734475   
8           True          0.471853       0.607622        0.524013   
            False         0.208849       0.546089        0.738849   
9           True          0.461607       0.587486        0.468759   
            False         0.264146       0.559017        0.735521   
10          True          0.466193       0.598067        0.501094   
            False         0.189225       0.541795        0.739515   
11          True          0.457860       0.580934        0.455445   
            False         0.240340       0.553373        0.737233   
12          True          0.460167       0.588289        0.481598   
            False         0.173324       0.537501        0.738374   
13          True          0.452522       0.572274        0.440323   
            False         0.217504       0.547510        0.737423   
14          True          0.455422       0.580389        0.465716   
            False         0.160209       0.535268        0.739515   
15          True          0.449291       0.566752        0.430433   
            False         0.196911       0.542955        0.738374   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.683578  
            False                        -0.423425  
2           True                         -0.618653  
            False                        -0.135409  
3           True                         -0.703664  
            False                        -0.346495  
4           True                         -0.745251  
            False                        -0.139074  
5           True                         -0.690220  
            False                        -0.265354  
6           True                         -0.751674  
            False                        -0.109628  
7           True                         -0.693301  
            False                        -0.222696  
8           True                         -0.750032  
            False                        -0.104916  
9           True                         -0.679582  
            False                        -0.179222  
10          True                         -0.738105  
            False                        -0.082394  
11          True                         -0.664979  
            False                        -0.145523  
12          True                         -0.730438  
            False                        -0.077841  
13          True                         -0.654828  
            False                        -0.124099  
14          True                         -0.719130  
            False                        -0.057675  
15          True                         -0.645359  
            False                        -0.098451

In [18]:
# Best model parameters

best_knn = knn_results.iloc[[
    knn_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.499423,0.646364,0.621113,-0.683578


In [60]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

display(knn_results_1, knn_results_2, knn_results_3)

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.429214       0.729216        0.703079   
            False         0.491925       0.705103        0.850567   
2           True          0.446824       0.725242        0.752026   
            False         0.329447       0.599798        0.873258   
3           True          0.376377       0.692769        0.603566   
            False         0.427306       0.648521        0.866775   
4           True          0.392462       0.701857        0.649595   
            False         0.255296       0.572566        0.866775   
5           True          0.356248       0.676156        0.542950   
            False         0.310488       0.593254        0.862885   
6           True          0.367621       0.685036        0.582496   
            False         0.174755       0.546133        0.859968   
7           True          0.341222       0.660486        0.495624   
            False         0.238976       0.566768        0.862237   
8           True          0.354093       0.674420        0.538412   
            False         0.104340       0.525495        0.856726   
9           True          0.330459       0.646197        0.466775   
            False         0.157399       0.540598        0.858671   
10          True          0.342557       0.661609        0.503079   
            False         0.091267       0.522482        0.857374   
11          True          0.327190       0.642569        0.448947   
            False         0.118768       0.529435        0.857374   
12          True          0.336913       0.654949        0.482334   
            False         0.057803       0.513382        0.855105   
13          True          0.325172       0.639977        0.438574   
            False         0.086641       0.520891        0.856402   
14          True          0.333462       0.650918        0.464830   
            False         0.042952       0.510204        0.855429   
15          True          0.318059       0.629186        0.420746   
            False         0.062729       0.515003        0.855754   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.973476  
            False                        -0.205525  
2           True                         -0.803485  
            False                         0.071255  
3           True                         -1.246103  
            False                        -0.039499  
4           True                         -1.170122  
            False                         0.034149  
5           True                         -1.282259  
            False                        -0.029604  
6           True                         -1.277375  
            False                        -0.005280  
7           True                         -1.261142  
            False                        -0.004820  
8           True                         -1.282356  
            False                        -0.007747  
9           True                         -1.250799  
            False                        -0.007933  
10          True                         -1.273901  
            False                         0.005241  
11          True                         -1.205633  
            False                        -0.009276  
12          True                         -1.250688  
            False                        -0.008596  
13          True                         -1.176405  
            False                        -0.004817  
14          True                         -1.210824  
            False                         0.001279  
15          True                         -1.163645  
            False                        -0.003814

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.434726       0.674297        0.674803   
            False         0.400683       0.630272        0.798688   
2           True          0.427913       0.660027        0.718635   
            False         0.181103       0.545906        0.822572   
3           True          0.394104       0.640517        0.568766   
            False         0.274667       0.571965        0.808136   
4           True          0.398407       0.642323        0.613123   
            False         0.098476       0.520691        0.813386   
5           True          0.384870       0.631225        0.520735   
            False         0.155382       0.533384        0.809186   
6           True          0.389610       0.635674        0.557218   
            False         0.063240       0.513248        0.814436   
7           True          0.371214       0.613458        0.474278   
            False         0.100243       0.520745        0.812598   
8           True          0.373766       0.616733        0.508136   
            False         0.043203       0.509270        0.814961   
9           True          0.365831       0.605725        0.451181   
            False         0.065138       0.512992        0.813123   
10          True          0.369971       0.611748        0.485827   
            False         0.019313       0.504438        0.815223   
11          True          0.365264       0.604605        0.436220   
            False         0.032622       0.506094        0.813386   
12          True          0.367526       0.608176        0.462467   
            False         0.013909       0.503210        0.814961   
13          True          0.359701       0.595949        0.418635   
            False         0.016516       0.503429        0.814436   
14          True          0.361661       0.599496        0.441995   
            False         0.005595       0.501252        0.814436   
15          True          0.355518       0.589289        0.403150   
            False         0.013834       0.502885        0.814436   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.870013  
            False                        -0.321187  
2           True                         -0.760851  
            False                        -0.014568  
3           True                         -1.082806  
            False                        -0.192327  
4           True                         -1.068373  
            False                        -0.054658  
5           True                         -1.066914  
            False                        -0.120900  
6           True                         -1.098607  
            False                        -0.027387  
7           True                         -1.049197  
            False                        -0.062467  
8           True                         -1.119156  
            False                        -0.012834  
9           True                         -1.016256  
            False                        -0.040193  
10          True                         -1.092333  
            False                         0.000464  
11          True                         -0.962178  
            False                        -0.022071  
12          True                         -1.041486  
            False                         0.000679  
13          True                         -0.941756  
            False                        -0.005282  
14          True                         -1.019827  
            False                        -0.000183  
15          True                         -0.913546  
            False                        -0.003944

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.689127       0.703541        0.703724   
            False         0.672060       0.697272        0.699586   
2           True          0.560815       0.650711        0.662621   
            False         0.515257       0.637930        0.652966   
3           True          0.665268       0.675007        0.674207   
            False         0.647446       0.679922        0.683586   
4           True          0.613848       0.667667        0.675034   
            False         0.562774       0.655127        0.667310   
5           True          0.664934       0.672780        0.671724   
            False         0.642784       0.675979        0.679724   
6           True          0.621419       0.661101        0.666207   
            False         0.565262       0.648736        0.659586   
7           True          0.659144       0.663622        0.662345   
            False         0.632759       0.668846        0.673379   
8           True          0.627243       0.659459        0.663448   
            False         0.583298       0.657696        0.667586   
9           True          0.656501       0.660140        0.658759   
            False         0.628001       0.666863        0.671724   
10          True          0.630138       0.659515        0.663172   
            False         0.577801       0.651371        0.660966   
11          True          0.644813       0.648146        0.646897   
            False         0.614933       0.657517        0.662621   
12          True          0.627116       0.652209        0.655172   
            False         0.569960       0.644983        0.654621   
13          True          0.647114       0.650351        0.649103   
            False         0.610816       0.654999        0.660414   
14          True          0.632716       0.654053        0.656276   
            False         0.577475       0.647393        0.656552   
15          True          0.650385       0.653331        0.651862   
            False         0.615725       0.658416        0.663724   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.188773  
            False                        -0.202878  
2           True                         -0.239732  
            False                        -0.195249  
3           True                         -0.303803  
            False                        -0.263845  
4           True                         -0.268417  
            False                        -0.209964  
5           True                         -0.310514  
            False                        -0.279059  
6           True                         -0.327508  
            False                        -0.270763  
7           True                         -0.343546  
            False                        -0.302151  
8           True                         -0.346290  
            False                        -0.255368  
9           True                         -0.356490  
            False                        -0.305518  
10          True                         -0.348695  
            False                        -0.288016  
11          True                         -0.403622  
            False                        -0.339063  
12          True                         -0.381845  
            False                        -0.313946  
13          True                         -0.393372  
            False                        -0.346371  
14          True                         -0.378093  
            False                        -0.316264  
15          True                         -0.383036  
            False                        -0.334788

In [20]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.429214,0.729216,0.703079,-0.973476


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.434726,0.674297,0.674803,-0.870013


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,
1,True,0.689127,0.703541,0.703724,-0.188773


#### 4. Support Vector Classifier

In [21]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  accuracy_score  \
kernel gamma oversampling                                            
linear 0.10  True          0.428619       0.579581        0.571945   
             False         0.000000       0.500000        0.730480   
rbf    0.10  True          0.084634       0.519190        0.737803   
             False         0.029801       0.507509        0.734475   
linear 0.01  True          0.428619       0.579581        0.571945   
             False         0.000000       0.500000        0.730480   
rbf    0.01  True          0.434135       0.602052        0.648407   
             False         0.050792       0.510803        0.734094   

                           explained_variance_score  
kernel gamma oversampling                            
linear 0.10  True                         -0.949189  
             False                         0.000000  
rbf    0.10  True                         -0.008115  
             False                         0.008894  
linear 0.01  True                         -0.949189  
             False                         0.000000  
rbf    0.01  True                         -0.750245  
             False                        -0.010248

In [22]:
# Best model parameters

best_svc = svc_results.iloc[[
    svc_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,True,0.434135,0.602052,0.648407,-0.750245


In [23]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [24]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()[DECISION_METRIC].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,True,0.414728,0.67027,0.8,-0.594108


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
linear,0.1,True,0.329036,0.571256,0.586877,-1.330589


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,
rbf,0.01,False,0.639119,0.675562,0.679448,-0.274921


#### 5. Naive Bayes Classifier

In [25]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.373249,0.558792,0.624156,-0.884740
False,0.335659,0.558044,0.674560,-0.639983


In [26]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.373249,0.558792,0.624156,-0.88474


In [27]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [28]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.330198,0.626599,0.645381,-1.399319


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.291533,0.53156,0.554068,-1.504631


,f1_score,roc_auc_score,accuracy_score,explained_variance_score
oversampling,,,,
True,0.467552,0.588102,0.601655,-0.455158


#### 6. Decision Tree

In [29]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  accuracy_score  \
max_depth oversampling                                            
3         True          0.407376       0.548584        0.515264   
          False         0.090122       0.512900        0.725915   
5         True          0.340102       0.545527        0.625678   
          False         0.114397       0.518004        0.726486   
7         True          0.323715       0.541309        0.643272   
          False         0.204995       0.530143        0.709843   

                        explained_variance_score  
max_depth oversampling                            
3         True                         -1.005036  
          False                        -0.105380  
5         True                         -0.837128  
          False                        -0.120899  
7         True                         -0.794798  
          False                        -0.319414

In [30]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
3,True,0.407376,0.548584,0.515264,-1.005036


In [31]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [32]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
5,True,0.331124,0.625771,0.673906,-1.29793


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
7,True,0.278113,0.540208,0.639895,-1.238409


,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,oversampling,,,,
3,True,0.539619,0.600186,0.606897,-0.545729


#### 7. Random Forest

In [33]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  accuracy_score  \
max_depth n_estimators oversampling                                            
3         3            True          0.397454       0.555896        0.566239   
                       False         0.051112       0.508871        0.730766   
          5            True          0.384618       0.553634        0.588398   
                       False         0.027973       0.503915        0.729529   
          7            True          0.392360       0.562141        0.596101   
                       False         0.024556       0.503776        0.730195   
5         3            True          0.354520       0.543363        0.606847   
                       False         0.084225       0.513525        0.728768   
          5            True          0.383721       0.565595        0.623966   
                       False         0.059132       0.508856        0.728768   
          7            True          0.370769       0.562583        0.638231   
                       False         0.056195       0.509918        0.731241   
7         3            True          0.363027       0.555538        0.629957   
                       False         0.152309       0.525619        0.725725   
          5            True          0.365460       0.563167        0.648692   
                       False         0.117996       0.518983        0.726866   
          7            True          0.353067       0.562875        0.666857   
                       False         0.083962       0.513485        0.728959   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -1.020717  
                       False                        -0.036169  
          5            True                         -1.000160  
                       False                        -0.028305  
          7            True                         -0.948201  
                       False                        -0.020710  
5         3            True                         -0.955254  
                       False                        -0.078135  
          5            True                         -0.869329  
                       False                        -0.058338  
          7            True                         -0.826637  
                       False                        -0.036366  
7         3            True                         -0.863299  
                       False                        -0.157053  
          5            True                         -0.779633  
                       False                        -0.120092  
          7            True                         -0.687852  
                       False                        -0.076662

In [34]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
5,5,True,0.383721,0.565595,0.623966,-0.869329


In [35]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)

In [36]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,5,True,0.368739,0.644008,0.758509,-0.886857


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,5,True,0.27565,0.544248,0.677953,-1.088363


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,7,True,0.590591,0.618884,0.621241,-0.520768


#### 8. Gradient Boosting

In [37]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  accuracy_score  \
max_depth n_estimators oversampling                                            
3         3            True          0.353651       0.552765        0.630052   
                       False         0.000000       0.500000        0.730480   
          5            True          0.356908       0.555510        0.634237   
                       False         0.001408       0.500289        0.730575   
          7            True          0.364988       0.559862        0.641274   
                       False         0.004207       0.500862        0.730766   
5         3            True          0.322218       0.549186        0.664384   
                       False         0.001423       0.500227        0.730480   
          5            True          0.308086       0.549344        0.678174   
                       False         0.009757       0.501940        0.731051   
          7            True          0.314920       0.553635        0.682834   
                       False         0.016775       0.503162        0.731146   
7         3            True          0.314533       0.549069        0.673704   
                       False         0.002841       0.500713        0.730861   
          5            True          0.302136       0.550702        0.689396   
                       False         0.024717       0.504450        0.731146   
          7            True          0.282224       0.546701        0.696624   
                       False         0.040788       0.507993        0.732382   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -0.826885  
                       False                         0.000000  
          5            True                         -0.820997  
                       False                        -0.000307  
          7            True                         -0.811042  
                       False                        -0.000903  
5         3            True                         -0.679571  
                       False                        -0.001027  
          5            True                         -0.593991  
                       False                        -0.002849  
          7            True                         -0.575443  
                       False                        -0.007132  
7         3            True                         -0.633578  
                       False                         0.000910  
          5            True                         -0.531986  
                       False                        -0.013410  
          7            True                         -0.471852  
                       False                        -0.015869

In [38]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
3,7,True,0.364988,0.559862,0.641274,-0.811042


In [39]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)

In [40]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()[DECISION_METRIC].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,7,True,0.371901,0.644666,0.770502,-0.802193


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
3,7,True,0.30588,0.56181,0.661155,-1.138283


,,,f1_score,roc_auc_score,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,
7,7,True,0.605705,0.634791,0.637517,-0.454528


### Results

In [41]:
#display('logistic', best_log)
#display('lda', best_lda)
#display('knn', best_knn)
#display('svc', best_svc)
#display('naive_bayes', best_nb)
#display('decision_tree', best_dt)
#display('random_forest', best_rf)
#display('gradient_boosting', best_gb)

In [42]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [43]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
#display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
#display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
#display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
#display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
#display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
#display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
#display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  accuracy_score  \
n_neighbors oversampling                                            
1           True          0.429214       0.729216        0.703079   
            True          0.434726       0.674297        0.674803   
            True          0.689127       0.703541        0.703724   

                          explained_variance_score  
n_neighbors oversampling                            
1           True                         -0.973476  
            True                         -0.870013  
            True                         -0.188773

In [44]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.579228,0.621144
1,lda,0.581468,0.625879
2,knn,0.646364,0.700479
3,svc,0.602052,0.636241
4,naive_bayes,0.558792,0.578915
5,decision_tree,0.548584,0.585969
6,random_forest,0.565595,0.599224
7,gradient_boosting,0.559862,0.611260


### Conclusions

In [45]:
# Save dataframe
# results.to_pickle("results_fine_no_metadata.pkl")

In [46]:
results.mean(axis=0)

auc_best               0.580243
auc_expert_weighted    0.619889
dtype: float64

In [47]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.646364
auc_expert_weighted    0.700479
Name: 2, dtype: object

In [48]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.646364
auc_expert_weighted    0.700479
Name: 2, dtype: object